# ChemicalMotifIdentifier

**To reviewers:** As we are keeping these codes private untill acceptance of the work, the PyPi installation of the package will not work. Please follow the instructions below to install all packages and dependencies.


In [ ]:
%%capture 
! pip install gdown

# Downling private packages
import gdown

gdown.download_folder(
    "https://drive.google.com/drive/folders/1jqf5LNeN-SDBuXJinHIlZa54YKbkAk6y?usp=drive_link",
    output="./",
    quiet=True,
)


! cd _additional_packages/Polya && pip install -e .  && cd .. 
! cd _additional_packages/NshellFinder && pip install -e .  && cd .. 
! cd _additional_packages/Simplex && pip install -e .  && cd .. 

# # Installing chemicalmotifidentifier
! cd .. && pip install -e . 

! pip install rich

This is a quick tutorial on:
1. Analytically obtaining a pattern inventory for a ternary system (CrCoNi) in the fcc crystal structure. 
1. Obtaining the pattern inventory with ML and creating a physically constrained embedding space from which we can compute dissimilarities between motifs. 
1. Computing dissimilarity between motifs in atomistic data.

In [ ]:
from polya import Polya
from rich import print
from sympy import init_printing, symbols  # For latex formatting
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

%matplotlib inline

from _frameworks import INPUT_GDOWN_LINK

# Downloading neceassry inputs (model weights, dump files, ...)
import gdown

os.makedirs("data/", exist_ok=True)
gdown.download_folder(INPUT_GDOWN_LINK, output="data/", quiet=True)

## 1. Ternary system pattern inventory

In [ ]:
init_printing()

pl = Polya(graph_name="fcc")

ntypes = 3
p_g, nms = pl.get_gt(ntypes=ntypes)

# Replacing t1, t2 and t3 with Cr, Co, Ni.
p_g = p_g.subs(
    {
        symbols("t1"): symbols("Cr"),
        symbols("t2"): symbols("Co"),
        symbols("t3"): symbols("Ni"),
    }
)

print(
    "The pattern inventory for the fcc first coordination polyhedron of the CrCoNi system is given by: \n"
)

p_g

In [ ]:
print(
    f"\n We have a total of {nms} distinct coordination polyhedron, which in turns yield {nms*ntypes} distinct local chemical motifs. "
)

## 2. Model expressivity on the fcc ternary synthetic dataset and physically constrained embedding space

In [ ]:
from _frameworks import SyntheticChemicalMotifIdentifier

eca = SyntheticChemicalMotifIdentifier(crystal_structure="fcc")
df = eca.predict(
    root="data/synthetic/fcc_graph_datasets/",
    skeleton_graph_path="data/inputs_doi-10.48550-arXiv.2311.01545/fcc_1nn.pt",
    atom_types_paths=[
        "data/inputs_doi-10.48550-arXiv.2311.01545/fcc_nelement3_generators.pt"
    ],
    nelement=3,
)
os.makedirs("data/synthetic/outputs/", exist_ok=True)
df.to_pickle("data/synthetic/outputs/df_fcc.pkl")

In [ ]:
# Obtaining the ML pattern inventory
df = pd.read_pickle("data/synthetic/outputs/df_fcc.pkl")

shell_concentrations, counts = (
    np.array(list(df.shell_concentration)),
    np.array(list(df.counts)),
)
unique_concentrations, counts = np.unique(
    shell_concentrations, axis=0, return_inverse=True
)

pattern = {
    tuple(unique_concentrations[i]): counts[i]
    for i in range(len(unique_concentrations))
}
print("The machine learning pattern inventory is given by:")
print(pattern)
print(f"We have a total of {len(counts)} distinct 1CP.")

## 3. Chemical motif identification in atomistic data

In [ ]:
from _frameworks import MonteCarloChemicalMotifIdentifier

dump_files = [
    f"data/inputs_doi-10.48550-arXiv.2311.01545/dumps/ordered_relaxation_20_{i}_300K.dump"
    for i in range(1, 5 + 1)
]

eca = MonteCarloChemicalMotifIdentifier(crystal_structure="fcc")
df = eca.predict(root="data/mc/outputs/eca_id/300K/", dump_file=dump_files)
kl = eca.get_kl(df)
df.to_pickle("data/mc/outputs/eca_id/300K/df_microstates.pkl")

Let's compute the dissimilarity between motif $\mathcal{M}_0$ and $\mathcal{M}_1$ shown below: 

In [ ]:
from chemicalmotifidentifier import Plot

plot = Plot(
    structure="fcc",
    graph_folder="data/inputs_doi-10.48550-arXiv.2311.01545/graph_plot_templates",
)
plot.set_colors(np.array(["#FED700", "#21B0FE", "#FE218B"]))
plot.set_node_size(4)
plot.set_width(0.2)

for i in [0, 1]:
    df_row = df.iloc[i]

    types_with_central_atom = np.concatenate(
        ([df_row.central_atom_type], np.array(list(df_row.shell_atomic_types)))
    )

    fig, ax = plt.subplots(figsize=(1, 1))
    plot.plot_ms(new_atom_types=types_with_central_atom)
    plt.show()

In [ ]:
from chemicalmotifidentifier import Dissimilarity

phys_emb_i = np.concatenate(([df.iloc[0].central_atom_type], df.iloc[0].shell_phys_emb))
phys_emb_j = np.concatenate(([df.iloc[1].central_atom_type], df.iloc[1].shell_phys_emb))

(
    central_atom_dissim,
    concentration_dissim,
    structural_dissim,
) = Dissimilarity().get_separate_dissimilarities(3, phys_emb_j, phys_emb_i)

weights = np.array([12.0, 24.0, 24.0])
weights *= 1 / np.sum(weights)
d_ij = (
    weights[0] * central_atom_dissim
    + weights[1] * concentration_dissim / 12
    + weights[2] * structural_dissim
)


print(f"The dissimilarity between these two motifs is: {d_ij}. \n 
Before weighthing based on the number of bonds of each structure, they have a central atom dissimilarity of {central_atom_dissim} since they are the same. Their chemical composition dissimilarity is of {concentration_dissim / 12}/12 because only two atoms are swapped. And their structural dissimilarity is of {structural_dissim}.")